<a href="https://colab.research.google.com/github/autheblanc/skills-introduction-to-github/blob/main/1201_prediction_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import h5py
with h5py.File('/content/1201_prediction_analysis.h5', 'r') as f:
    occupancy_matrix = f['track_occupancy'][:]
    tracks_matrix = f['tracks'][:]

print(occupancy_matrix.shape)
print(tracks_matrix.shape)

(293, 1)
(1, 2, 3, 293)


In [2]:
with h5py.File( '/content/1201_prediction_analysis.h5', 'r') as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

print("===filename===")
print('/content/1201_prediction_analysis.h5')
print()

print("===HDF5 datasets===")
print(dset_names)
print()

print("===locations data shape===")
print(locations.shape)
print()

print("===nodes===")
for i, name in enumerate(node_names):
    print(f"{i}: {name}")
print()

===filename===
/content/1201_prediction_analysis.h5

===HDF5 datasets===
['edge_inds', 'edge_names', 'instance_scores', 'labels_path', 'node_names', 'point_scores', 'provenance', 'track_names', 'track_occupancy', 'tracking_scores', 'tracks', 'video_ind', 'video_path']

===locations data shape===
(293, 3, 2, 1)

===nodes===
0: snout
1: eye
2: ear



In [3]:
frame_count, node_count, _, instance_count = locations.shape

print("frame count:", frame_count)
print("node count:", node_count)
print("instance count:", instance_count)

frame count: 293
node count: 3
instance count: 1


In [4]:
!pip uninstall -qqq -y opencv-python opencv-contrib-python
!pip install -qqq "sleap[pypi]>=1.3.3"


In [5]:
# Test video:
!wget https://storage.googleapis.com/sleap-data/reference/flies13/190719_090330_wt_18159206_rig1.2%4015000-17560.mp4

# Test video labels (from predictions/not necessary for inference benchmarking):
!wget https://storage.googleapis.com/sleap-data/reference/flies13/190719_090330_wt_18159206_rig1.2%4015000-17560.slp

# Bottom-up model:
# !wget https://storage.googleapis.com/sleap-data/reference/flies13/bu.210506_230852.multi_instance.n%3D1800.zip

# Top-down model (two-stage):
!wget https://storage.googleapis.com/sleap-data/reference/flies13/centroid.fast.210504_182918.centroid.n%3D1800.zip
!wget https://storage.googleapis.com/sleap-data/reference/flies13/td_fast.210505_012601.centered_instance.n%3D1800.zip

--2024-12-02 17:56:06--  https://storage.googleapis.com/sleap-data/reference/flies13/190719_090330_wt_18159206_rig1.2%4015000-17560.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.207, 74.125.199.207, 74.125.20.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85343812 (81M) [video/mp4]
Saving to: ‘190719_090330_wt_18159206_rig1.2@15000-17560.mp4.1’

190719_090330_wt_18 100%[===================>]  81.39M  38.9MB/s    in 2.1s    

2024-12-02 17:56:08 (38.9 MB/s) - ‘190719_090330_wt_18159206_rig1.2@15000-17560.mp4.1’ saved [85343812/85343812]

--2024-12-02 17:56:08--  https://storage.googleapis.com/sleap-data/reference/flies13/190719_090330_wt_18159206_rig1.2%4015000-17560.slp
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.207, 74.125.199.207, 74.125.20.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|1

In [6]:
!ls -lah

total 297M
drwxr-xr-x 1 root root 4.0K Dec  2 17:56  .
drwxr-xr-x 1 root root 4.0K Dec  2 17:38  ..
-rw-r--r-- 1 root root  26K Dec  2 17:43  1201_prediction_analysis.h5
-rw-r--r-- 1 root root  82M May 20  2021  190719_090330_wt_18159206_rig1.2@15000-17560.mp4
-rw-r--r-- 1 root root  82M May 20  2021  190719_090330_wt_18159206_rig1.2@15000-17560.mp4.1
-rw-r--r-- 1 root root 1.6M May 20  2021  190719_090330_wt_18159206_rig1.2@15000-17560.slp
-rw-r--r-- 1 root root 1.6M May 20  2021  190719_090330_wt_18159206_rig1.2@15000-17560.slp.1
-rw-r--r-- 1 root root  59M Dec  2 17:43  241125_012328.single_instance.zip
-rw-r--r-- 1 root root 6.1M May 20  2021 'centroid.fast.210504_182918.centroid.n=1800.zip'
-rw-r--r-- 1 root root 6.1M May 20  2021 'centroid.fast.210504_182918.centroid.n=1800.zip.1'
drwxr-xr-x 4 root root 4.0K Nov 25 19:13  .config
-rw-r--r-- 1 root root 1.4M Dec  2 17:43  EC01_results_2024-11-08T10_20_2_Trial_23.avi
drwxr-xr-x 1 root root 4.0K Nov 25 19:13  sample_data
-rw-r--r-- 

In [18]:
import sleap

# This prevents TensorFlow from allocating all the GPU memory, which leads to issues on
# some GPUs/platforms:
sleap.disable_preallocation()

# This would hide GPUs from the TensorFlow altogether:
# sleap.use_cpu_only()

# Print some info:
sleap.versions()
sleap.system_summary()

SLEAP: 1.3.4
TensorFlow: 2.8.4
Numpy: 1.22.4
Python: 3.10.12
OS: Linux-6.1.85+-x86_64-with-glibc2.35
GPUs: None detected.


In [8]:
# Videos can be represented agnostic to the backend format
video = sleap.load_video("/content/EC01_results_2024-11-08T10_20_2_Trial_23.avi")

# sleap.Video objects have a numpy-like interface:
print(video.shape)

# And we can load images in the video using array indexing:
imgs = video[:4]
print(imgs.shape, imgs.dtype)

(503, 480, 640, 3)
(4, 480, 640, 3) uint8


In [9]:
predictor = sleap.load_model("/content/241125_012328.single_instance.zip")

/usr/local/lib/python3.10/dist-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [10]:
labels = predictor.predict(video)
labels

Output()

Labels(labeled_frames=503, videos=1, skeletons=1, tracks=0)

In [11]:
labels.videos

[Video(backend=MediaVideo(filename='/content/EC01_results_2024-11-08T10_20_2_Trial_23.avi', grayscale=False, bgr=True, dataset='', input_format='channels_last'))]

In [12]:
labels.skeletons

[Skeleton(name='Skeleton-3', description='None', nodes=['snout', 'eye', 'ear'], edges=[], symmetries=[])]

In [13]:
labeled_frame = labels
labeled_frame

Labels(labeled_frames=503, videos=1, skeletons=1, tracks=0)

In [14]:
labeled_frame.instances

<bound method Labels.instances of Labels(labeled_frames=503, videos=1, skeletons=1, tracks=0)>

In [15]:
instance = labeled_frame  # shortcut for labeled_frame.instances[0]
instance

Labels(labeled_frames=503, videos=1, skeletons=1, tracks=0)

In [16]:
pts = instance.numpy()
print(pts)

[[[[620.3184  267.8072 ]
   [600.14667 248.22794]
   [568.1183  239.43674]]]


 [[[620.3187  267.79874]
   [600.1289  248.23192]
   [568.0957  239.46967]]]


 [[[620.3054  267.83078]
   [600.1061  248.2154 ]
   [568.0035  239.3862 ]]]


 ...


 [[[623.9064  267.9637 ]
   [603.66077 251.97357]
   [567.9694  243.95343]]]


 [[[623.89777 267.9253 ]
   [603.63385 251.92723]
   [567.8862  243.95735]]]


 [[[623.84393 267.9788 ]
   [603.6308  251.96356]
   [567.92523 243.97899]]]]


In [39]:
import numpy as np
for i in range (0,503):
  A= np.matrix(pts[i])
  B = A[:,0]
  print(B.transpose())

[[620.3184  600.14667 568.1183 ]]
[[620.3187 600.1289 568.0957]]
[[620.3054 600.1061 568.0035]]
[[623.76917 600.2727  568.1795 ]]
[[623.82477 603.52637 568.3268 ]]
[[623.83716 603.5658  568.4595 ]]
[[623.8185 603.5552 568.4156]]
[[623.82544 603.5541  568.39124]]
[[623.7763  603.55194 568.3582 ]]
[[623.7601  603.54724 568.36066]]
[[623.72723 603.5478  568.39197]]
[[623.7493  603.5357  568.38336]]
[[623.8116  603.55963 568.33136]]
[[623.8074  600.35986 568.28357]]
[[623.8012 600.3737 568.3147]]
[[623.85126 603.59814 568.408  ]]
[[623.793  603.5612 568.382 ]]
[[623.82477 603.5701  568.37396]]
[[623.82306 603.60236 568.40594]]
[[623.8831  603.64813 568.44366]]
[[623.8694  603.60785 568.4505 ]]
[[623.87396 603.6268  568.46893]]
[[623.87286 603.65466 571.5026 ]]
[[623.8752 603.6705 571.5224]]
[[623.8865 603.6947 571.5576]]
[[623.92487 603.7103  571.49084]]
[[623.8917  603.68524 571.45996]]
[[623.92303 603.70325 571.50366]]
[[623.9058 603.7156 571.57  ]]
[[623.8816  603.69354 571.57214]]
[[62

In [41]:
import numpy as np
for i in range(0,503):
  A=np.matrix(pts[i])
  C = A[:,1]
  print(C.transpose())

[[267.8072  248.22794 239.43674]]
[[267.79874 248.23192 239.46967]]
[[267.83078 248.2154  239.3862 ]]
[[267.8276  248.34464 239.66011]]
[[267.84003 248.41641 239.83214]]
[[267.82632 251.47734 239.96764]]
[[267.83383 251.50633 239.98473]]
[[267.79535 251.52124 239.99461]]
[[267.77213 248.47472 239.96387]]
[[267.75504 248.44441 239.96141]]
[[267.7518  248.42018 239.9179 ]]
[[267.7871  248.42233 239.90622]]
[[267.82736 248.4202  239.89938]]
[[267.81824 248.32774 239.8008 ]]
[[267.8595  248.3258  239.82733]]
[[267.82922 248.3954  239.89377]]
[[267.78513 248.37411 239.87369]]
[[267.74274 248.35063 239.9004 ]]
[[267.75964 248.3519  239.86862]]
[[267.80133 248.37645 239.87086]]
[[267.7948  248.41025 239.89171]]
[[267.7999  251.50203 239.97728]]
[[267.7835  248.42506 239.98479]]
[[267.7673  248.43755 239.9809 ]]
[[267.82162 251.4996  239.99702]]
[[267.82938 248.43881 239.97781]]
[[267.88998 248.42834 239.93036]]
[[267.7366  248.37381 239.9052 ]]
[[267.8389  248.45071 239.92583]]
[[267.82043 24